In [1]:
import numpy as np
from numpy import pi
from qiskit import QuantumCircuit, transpile, assemble, Aer, IBMQ
from qiskit.providers.ibmq import least_busy
from qiskit.tools.monitor import job_monitor
from qiskit.visualization import plot_histogram, plot_bloch_multivector
import qiskit.quantum_info as qi
from qiskit.providers.aer import AerSimulator
from qiskit.providers.aer.noise import NoiseModel, amplitude_damping_error
from qiskit import transpile
import matplotlib.pyplot as plt
from qiskit.providers.ibmq import least_busy
from qiskit import ClassicalRegister, QuantumRegister

In [2]:
n=8 # 2^number of qubits

In [3]:
#right shift
rshift=np.eye(n,k=-1)
rshift[0][-1]=1

#left shift
lshift=np.eye(n,k=1)
lshift[-1][0]=1

print(rshift,"\n")
print(lshift)

[[0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]] 

[[0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0.]]


In [4]:
pr=qi.Operator(r)
pl=qi.Operator(l)

NameError: name 'r' is not defined

In [341]:
def testToff(a,b,c):
    qc = QuantumCircuit(3)
    if a:
        qc.x(0)
    if b:
        qc.x(1)
    if c:
        qc.x(2)
    qc.ccx(0,1,2)
#     for i in range(3):
#         qc.measure(i,i)
    return qc

tm=testToff(1,0,1)
tm.draw()

┌───┐     
q_0: ┤ X ├──■──
     └───┘  │  
q_1: ───────■──
     ┌───┐┌─┴─┐
q_2: ┤ X ├┤ X ├
     └───┘└───┘

In [342]:
for a in range(2):
    for b in range(2):
        for c in range(2):
            print("input: "+str(a)+str(b)+str(c))
            t=testToff(a,b,c)
            t.save_statevector()
            
            sv_sim = Aer.get_backend('aer_simulator')
            qobj = assemble(t)  # Assemble circuit into a Qobj that can be run
            sv = sim.run(qobj).result().get_statevector()  # Do the simulation, returning the state vector
            print(sv)
            print("\n")

input: 000
[1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]


input: 001
[0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j]


input: 010
[0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]


input: 011
[0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j]


input: 100
[0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]


input: 101
[0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j]


input: 110
[0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j]


input: 111
[0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]




In [343]:
toff=np.eye(8)
toff[3][-1]=1
toff[-1][3]=1
toff[-1][-1]=0
toff[3][3]=0
print(toff)

[[1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]]


In [344]:
def testToffMatrix(a,b,c):
    qc = QuantumCircuit(3)
    if a:
        qc.x(0)
    if b:
        qc.x(1)
    if c:
        qc.x(2)
    qc.unitary(toff, [0, 1, 2], label='Toffoli')
    return qc

tf=testToffMatrix(1,0,1)
tf.draw()

┌───┐┌──────────┐
q_0: ┤ X ├┤0         ├
     └───┘│          │
q_1: ─────┤1 Toffoli ├
     ┌───┐│          │
q_2: ┤ X ├┤2         ├
     └───┘└──────────┘

In [345]:
for i in range(2):
    for j in range(2):
        for k in range(2):
            print("input: "+str(i)+str(j)+str(k))
            toffUni=testToffMatrix(i,j,k)
            
            sv_sim = Aer.get_backend('aer_simulator')
            toffUni.save_statevector()
            qobj = assemble(toffUni) 
            sv = sim.run(qobj).result().get_statevector() 
            print(sv)
            print("\n")

input: 000
[1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]


input: 001
[0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j]


input: 010
[0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]


input: 011
[0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j]


input: 100
[0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]


input: 101
[0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j]


input: 110
[0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j]


input: 111
[0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]


